In [36]:
%load_ext autoreload
%autoreload 2
import matplotlib
matplotlib.use('TkAgg')
#%matplotlib inline
import BPV
import pattern_manipulation as pm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import rcParams
import pickle
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math
import gc

matplotlib.style.use('ggplot')
rcParams['text.latex.unicode'] = True
rcParams['text.usetex'] = True
rcParams['text.latex.preamble'] = r'\usepackage{amsthm}', r'\usepackage{amsmath}', r'\usepackage{amssymb}', \
                                   r'\usepackage{amsfonts}'
#rcParams['text.latex.preamble']
#rcParams['text.latex.preamble'] = r'\usepackage{amsthm}'

Wmin = 0.005
Wmax = 0.05
Wnsteps = 10
Wstep = (Wmax - Wmin)/Wnsteps
Wrange = np.arange(Wmin,Wmax,Wstep)

Nmin = 5
Nmax = 15
Nnsteps = 10
Nstep = (Nmax - Nmin)/Nnsteps
Nrange = np.arange(Nmin,Nmax,Nstep, dtype='int')

nproblems = Wnsteps*Nnsteps

epsilon = 0.2

data = BPV.Data()
data.read_csv("pixel.dist.csv",False)
data.df.sort_index(by="p",inplace=True,ascending=False)
data.df.set_index(pd.Index([j for j in range(len(data.df))]), inplace=True)
data = data.data_head(100)
#data.order_by_p()
data.df['plog1onp'].sum()
#Nrange

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2.3209944808681966

In [2]:
#CALCULATE EPSILON-SOLUTIONS

eps_prbls = {}
it = 1
for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    W = Wrange[i]
    N = Nrange[j]
    print("\n\nSolving problem {0} out of {1}, for W = {2} and N = {3} i = {4}  j = {5}".format(it, nproblems, W, N,i,j))
    
    data.quantize_entropy(N, epsilon)

    prbl_eps = BPV.BPV("decgraphW",data,N,W,time_solver=True,\
                          use_quantized_entropy=True)
    prbl_eps.solve()
    prbl_eps.pprint_solution()
    del prbl_eps.predecessor
    eps_prbls[(i,j)] = prbl_eps
    
    gc.collect()
    it += 1
    



Solving problem 1 out of 100, for W = 0.005 and N = 5 i = 0  j = 0

Solver =  decgraphW 
Entropy =  0.034499234721 
Cardinality =  5 
Rate =  0.00499703910228
Solution time in seconds =  3.975245857000118


Solving problem 2 out of 100, for W = 0.005 and N = 6 i = 0  j = 1

Solver =  decgraphW 
Entropy =  0.0353772207993 
Cardinality =  6 
Rate =  0.00499717265653
Solution time in seconds =  5.194167743989965


Solving problem 3 out of 100, for W = 0.005 and N = 7 i = 0  j = 2

Solver =  decgraphW 
Entropy =  0.0360537568807 
Cardinality =  7 
Rate =  0.0049986551087
Solution time in seconds =  6.2253748739894945


Solving problem 4 out of 100, for W = 0.005 and N = 8 i = 0  j = 3

Solver =  decgraphW 
Entropy =  0.0367390497629 
Cardinality =  8 
Rate =  0.00499941636793
Solution time in seconds =  7.99707651900826


Solving problem 5 out of 100, for W = 0.005 and N = 9 i = 0  j = 4

Solver =  decgraphW 
Entropy =  0.0373670720953 
Cardinality =  9 
Rate =  0.00499925610283
Solution

In [20]:
zeroindexes = (Wentropies == 0).nonzero()
totzeros = len(zeroindexes[0])
zeroindexes = zip(zeroindexes[0], zeroindexes[1])
for i,j in zeroindexes:
    print(i,j,totzeros)

In [3]:
#SAVE EPSILON SOLUTIONS
f = open("/home/renato/tesi/code/data/plot12.eps_prbls",r'wb')
pickle.dump(eps_prbls,f)


In [24]:
#LOAD EPSILON SOLUTIONS
f = open("/home/renato/tesi/code/data/plot12.eps_prbls",r'rb')
eps_prbls = pickle.load(f)


In [27]:
eps_prbls[0,0].solution_entropy

0.049771874846470339

In [26]:
#RECALCULATE SOLUTION ATTRIBUTES (FOR CONVERTING TO LOG 2)

for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    eps_prbls[i,j].recalculate_solution_attributes()

In [5]:
#CALCULATE EXACT SOLUTIONS

Pentropies = np.zeros((Wnsteps, Nnsteps))
Prates = np.zeros((Wnsteps, Nnsteps))
Pcardinalities = np.zeros((Wnsteps, Nnsteps), dtype='int')
Ptimes = np.zeros((Wnsteps, Nnsteps))
Pindexes = {}


it = 1
for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    W = Wrange[i]
    N = Nrange[j]
    print("\n\nSolving problem {0} out of {1}, for W = {2} and N = {3} i = {4}  j = {5}".format(it, nproblems, W, N,i,j))

    Pprbl_exact = BPV.BPV("glpk",data,N,W,time_solver=True)
    Pprbl_exact.solve()
    Pprbl_exact.pprint_solution()
    
    Pentropies[i,j] = Pprbl_exact.solution_entropy
    Prates[i,j] = Pprbl_exact.solution_rate
    Pcardinalities[i,j] = Pprbl_exact.solution_cardinality
    Pindexes[(i,j)] = Pprbl_exact.solution.index
    Ptimes[(i,j)] = Pprbl_exact.solution_time
    
    gc.collect()
    it += 1
    



Solving problem 1 out of 100, for W = 0.005 and N = 5 i = 0  j = 0

Solver =  glpk 
Entropy =  0.0345318781306 
Cardinality =  5 
Rate =  0.00499981703068
Solution time in seconds =  0.1528092340013245


Solving problem 2 out of 100, for W = 0.005 and N = 6 i = 0  j = 1

Solver =  glpk 
Entropy =  0.0353918871971 
Cardinality =  6 
Rate =  0.00499984374153
Solution time in seconds =  0.058032970991916955


Solving problem 3 out of 100, for W = 0.005 and N = 7 i = 0  j = 2

Solver =  glpk 
Entropy =  0.0360608297481 
Cardinality =  7 
Rate =  0.00499980367525
Solution time in seconds =  0.17831089299579617


Solving problem 4 out of 100, for W = 0.005 and N = 8 i = 0  j = 3

Solver =  glpk 
Entropy =  0.0367519932321 
Cardinality =  8 
Rate =  0.00499691890346
Solution time in seconds =  0.04882588199689053


Solving problem 5 out of 100, for W = 0.005 and N = 9 i = 0  j = 4

Solver =  glpk 
Entropy =  0.0373813394978 
Cardinality =  9 
Rate =  0.00499846813275
Solution time in second

In [46]:
#SAVE EXACT SOLUTIONS
f1 = open("/home/renato/tesi/code/data/plot12.ex-entropies",r'wb')
f2 = open("/home/renato/tesi/code/data/plot12.ex-rates",r'wb')
f3 = open("/home/renato/tesi/code/data/plot12.ex-cardinalities",r'wb')
f4= open("/home/renato/tesi/code/data/plot12.ex-indexes",r'wb')
f5= open("/home/renato/tesi/code/data/plot12.ex-times",r'wb')

pickle.dump(Pentropies,f1)
pickle.dump(Prates,f2)
pickle.dump(Pcardinalities,f3)
pickle.dump(Pindexes,f4)
pickle.dump(Ptimes,f5)

In [3]:
#LOAD EXACT SOLUTIONS
f1 = open("/home/renato/tesi/code/data/plot12.ex-entropies",r'rb')
f2 = open("/home/renato/tesi/code/data/plot12.ex-rates",r'rb')
f3 = open("/home/renato/tesi/code/data/plot12.ex-cardinalities",r'rb')
f4 = open("/home/renato/tesi/code/data/plot12.ex-indexes",r'rb')
f5 = open("/home/renato/tesi/code/data/plot12.ex-times",r'rb')

Pentropies = pickle.load(f1)
Prates = pickle.load(f2)
Pcardinalities = pickle.load(f3)
Pindexes = pickle.load(f4)
Ptimes = pickle.load(f5)

In [28]:
#RESCALE ENTROPIES (FOR HAVING THEM IN LOG 2)
Pentropies *= np.log2(np.e)


In [29]:
Wentropies = np.zeros((Wnsteps, Nnsteps))
Wrates = np.zeros((Wnsteps, Nnsteps))
Wcardinalities = np.zeros((Wnsteps, Nnsteps), dtype='int')
Wtimes = np.zeros((Wnsteps, Nnsteps))
Wnnodes = np.zeros((Wnsteps, Nnsteps), dtype='int')
Wmultiplesols = np.zeros((Wnsteps, Nnsteps))
Windexes = {}

for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    Wentropies[i,j] = eps_prbls[i,j].solution_entropy
    Wrates[i,j] = eps_prbls[i,j].solution_rate
    Wcardinalities[i,j] = eps_prbls[i,j].solution_cardinality
    Wtimes[i,j] = eps_prbls[i,j].solution_time
    Windexes[i,j] = eps_prbls[i,j].solution.index
    Wnnodes[i,j] = eps_prbls[i,j].nnodes
    Wmultiplesols[i,j] = eps_prbls[i,j].multiple_solutions
#Wmultiplesols

In [31]:
# EPSILON SOLUTIONS
fig = plt.figure(figsize=(9,25))
#ax = fig.gca(projection='3d')
X, Y = np.meshgrid(Wrange, Nrange)
elev = 22
azimuth = -135
#elev = 40
#azimuth = 47
al = 0.7
cb_shrink = 0.1
cb_aspect = 10
label_size = 16
title_size = 20
title_rotation = 'horizontal'
title_x = 0.4
title_y = 0
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
colormap = cm.gnuplot
_dpi = 80

#gs = gridspec.GridSpec(2,7, width_ratios=[1,0.1,10,0.1,10,0.1,10], height_ratios=[1,1])

gs = gridspec.GridSpec(4,2, width_ratios=[1,100], height_ratios=[1,10,10,10])

title1 = plt.subplot(gs[0,:2])
cex_entropy = plt.subplot(gs[1,0])
ex_entropy = plt.subplot(gs[1,1], projection='3d') #exact entropy

cex_rate = plt.subplot(gs[2,0])
ex_rate = plt.subplot(gs[2,1], projection='3d') #exact rate

cex_card = plt.subplot(gs[3,0])
ex_card = plt.subplot(gs[3,1], projection='3d') #exact cardinality

title1.axis('off')
title1.text(title_x,title_y,"$\epsilon$-Solution",fontsize=title_size, rotation=title_rotation)

surf_e = ex_entropy.plot_surface(X, Y, Wentropies.T, rstride=1, cstride=1, alpha=al, cmap=colormap)
ex_entropy.set_xlabel('W',fontsize=label_size)
ex_entropy.set_ylabel('N',fontsize=label_size)
ex_entropy.zaxis.set_rotate_label(False)  # disable automatic rotation
ex_entropy.set_zlabel('Entropy',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ex_entropy.view_init(elev,azimuth)
plt.colorbar(surf_e, cax=cex_entropy)



surf_r = ex_rate.plot_surface(X, Y, Wrates.T, rstride=1, cstride=1, alpha=al, cmap=colormap)
ex_rate.set_xlabel('W',fontsize=label_size)
ex_rate.set_ylabel('N',fontsize=label_size)
ex_rate.zaxis.set_rotate_label(False)  # disable automatic rotation
ex_rate.set_zlabel('Rate',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ex_rate.view_init(elev,azimuth)
plt.colorbar(surf_r, cax=cex_rate)

surf_c = ex_card.plot_surface(X, Y, Wcardinalities.T, rstride=1, cstride=1, alpha=al, cmap=colormap)
ex_card.set_xlabel('W',fontsize=label_size)
ex_card.set_ylabel('N',fontsize=label_size)
ex_card.zaxis.set_rotate_label(False)  # disable automatic rotation
ex_card.set_zlabel('Cardinality',rotation='vertical',fontsize=label_size, horizontalalignment='right',)
ex_card.view_init(elev,azimuth)
plt.colorbar(surf_c, cax=cex_card)

fig.tight_layout()
#plt.savefig("/home/renato/tesi/testogit/img/epsilon-summary-epsilon.pdf",  bbox_inches='tight')
plt.show()

In [32]:
# EXACT SOLUTIONS
fig = plt.figure(figsize=(9,25))
#ax = fig.gca(projection='3d')
X, Y = np.meshgrid(Wrange, Nrange)
elev = 22
azimuth = -135
#elev = 40
#azimuth = 47
al = 0.7
cb_shrink = 0.1
cb_aspect = 10
label_size = 16
title_size = 20
title_rotation = 'horizontal'
title_x = 0.4
title_y = 0
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
colormap = cm.gnuplot
_dpi = 80

#gs = gridspec.GridSpec(2,7, width_ratios=[1,0.1,10,0.1,10,0.1,10], height_ratios=[1,1])

gs = gridspec.GridSpec(4,2, width_ratios=[1,100], height_ratios=[1,10,10,10])

title1 = plt.subplot(gs[0,:2])
cex_entropy = plt.subplot(gs[1,0])
ex_entropy = plt.subplot(gs[1,1], projection='3d') #exact entropy

cex_rate = plt.subplot(gs[2,0])
ex_rate = plt.subplot(gs[2,1], projection='3d') #exact rate

cex_card = plt.subplot(gs[3,0])
ex_card = plt.subplot(gs[3,1], projection='3d') #exact cardinality

title1.axis('off')
title1.text(title_x,title_y,"Exact Solution",fontsize=title_size, rotation=title_rotation)

surf_e = ex_entropy.plot_surface(X, Y, Pentropies.T, rstride=1, cstride=1, alpha=al, cmap=colormap)
ex_entropy.set_xlabel('W',fontsize=label_size)
ex_entropy.set_ylabel('N',fontsize=label_size)
ex_entropy.zaxis.set_rotate_label(False)  # disable automatic rotation
ex_entropy.set_zlabel('Entropy',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ex_entropy.view_init(elev,azimuth)
plt.colorbar(surf_e, cax=cex_entropy)



surf_r = ex_rate.plot_surface(X, Y, Prates.T, rstride=1, cstride=1, alpha=al, cmap=colormap)
ex_rate.set_xlabel('W',fontsize=label_size)
ex_rate.set_ylabel('N',fontsize=label_size)
ex_rate.zaxis.set_rotate_label(False)  # disable automatic rotation
ex_rate.set_zlabel('Rate',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ex_rate.view_init(elev,azimuth)
plt.colorbar(surf_r, cax=cex_rate)

surf_c = ex_card.plot_surface(X, Y, Pcardinalities.T, rstride=1, cstride=1, alpha=al, cmap=colormap)
ex_card.set_xlabel('W',fontsize=label_size)
ex_card.set_ylabel('N',fontsize=label_size)
ex_card.zaxis.set_rotate_label(False)  # disable automatic rotation
ex_card.set_zlabel('Cardinality',rotation='vertical',fontsize=label_size, horizontalalignment='right',)
ex_card.view_init(elev,azimuth)
plt.colorbar(surf_c, cax=cex_card)

fig.tight_layout()
#plt.savefig("/home/renato/tesi/testogit/img/epsilon-summary-exact.pdf",  bbox_inches='tight')
plt.show()

In [5]:
#CALCULATE GRADIENT, NOT INTERVAL MEASURE, RELATIVE ERROR
#ax = fig.gca(projection='3d')
#X, Y = np.meshgrid(Wrange, Nrange)
gx, gy = np.gradient(Pentropies)
grad = np.zeros((Wnsteps, Nnsteps))
not_interval_measure_ex =  np.zeros((Wnsteps, Nnsteps))
not_interval_measure_eps =  np.zeros((Wnsteps, Nnsteps))
relerr = np.zeros((Wnsteps,Nnsteps))
for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    grad[i,j] = np.sqrt(gx[i,j]**2 + gy[i,j]**2)
    not_interval_measure_ex[i,j] = BPV.not_interval_measure(Pindexes[i,j])
    not_interval_measure_eps[i,j] = BPV.not_interval_measure(Windexes[i,j])
    relerr[i,j] = (Pentropies[i,j] - Wentropies[i,j])/Pentropies[i,j]

In [19]:
#RELATIVE ERRROR
matplotlib.style.use('ggplot')

fig = plt.figure(figsize=(10,5))
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

X, Y = np.meshgrid(Wrange, Nrange)
elev = 25
azimuth = 57
al = 0.6
cb_shrink = 0.5
cb_aspect = 10
label_size = 16
#gradoffset = 5*relerr.min()
gradoffset = 0
gradlinew = 3
gradcolor = 'k'
ncountourlines = 7
ext3d = False
colormap = cm.gnuplot

def fmt1(x, pos):
    a = '{:.1e}'.format(x)
    return(a)

def fmt0(x, pos):
    a = '{:.0e}'.format(x)
    return(a)

relmin = relerr.min()
relmax = relerr.max()
ax = fig.add_subplot(1, 1, 1, projection='3d')
surf_e = ax.plot_surface(X, Y, relerr.T, rstride=1, cstride=1, alpha=al, \
                         vmin=relmin, vmax=relmax, cmap=colormap)
#cset = ax.contour(X, Y, grad.T, zdir='z', offset=gradoffset, colors=gradcolor, \
#                  levels=grcontourlevels, linewidths=gradlinew, extend3d=ext3d)
ax.set_xlabel('W')
ax.set_ylabel('N')
ax.set_zlim(0,relmax)
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('Relative Error',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
ax.zaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(fmt0))
cbar = fig.colorbar(surf_e,shrink=cb_shrink, aspect=cb_aspect, \
                    format=matplotlib.ticker.FuncFormatter(fmt1))

plt.show()

In [24]:
#NOT INTERVAL MEASURE EXACT
matplotlib.style.use('ggplot')

fig = plt.figure(figsize=(10,5))
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

X, Y = np.meshgrid(Wrange, Nrange)
elev = 35
azimuth = 31
al = 0.65
cb_shrink = 0.5
cb_aspect = 10
label_size = 16
gradoffset = 5*relerr.min()
gradlinew = 3
gradcolor = 'k'
ncountourlines = 7
ext3d = False
colormap = cm.gnuplot

mingr = grad.min()
maxgr = grad.max()
grcontourlevels = list(np.arange(mingr,maxgr,(maxgr-mingr)/ncountourlines ))

nimin = not_interval_measure_ex.min()
nimax = not_interval_measure_ex.max()
ax = fig.add_subplot(1, 1, 1, projection='3d')
surf_r = ax.plot_surface(X, Y, not_interval_measure_ex.T, rstride=1, cstride=1, alpha=al,\
                         vmin=nimin,vmax=nimax, cmap=colormap)
#cset = ax.contour(X, Y, grad.T, zdir='z', offset=gradoffset, colors=gradcolor, \
#                  levels=grcontourlevels, linewidths=gradlinew, extend3d=ext3d)
ax.set_xlabel('W')
ax.set_ylabel('N')
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('$\mathcal{I}(S)$',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
fig.colorbar(surf_r, shrink=cb_shrink, aspect=cb_aspect)

plt.show()

In [25]:
#NOT INTERVAL MEASURE EPSILON
matplotlib.style.use('ggplot')

fig = plt.figure(figsize=(10,5))
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

X, Y = np.meshgrid(Wrange, Nrange)
elev = 35
azimuth = 31
al = 0.65
cb_shrink = 0.5
cb_aspect = 10
label_size = 16
gradoffset = 5*relerr.min()
gradlinew = 3
gradcolor = 'k'
ncountourlines = 7
ext3d = False
colormap = cm.gnuplot

mingr = grad.min()
maxgr = grad.max()
grcontourlevels = list(np.arange(mingr,maxgr,(maxgr-mingr)/ncountourlines ))

nimin = not_interval_measure_eps.min()
nimax = not_interval_measure_eps.max()
ax = fig.add_subplot(1, 1, 1, projection='3d')
surf_r = ax.plot_surface(X, Y, not_interval_measure_eps.T, rstride=1, cstride=1, alpha=al,\
                         vmin=nimin,vmax=nimax, cmap=colormap)
#cset = ax.contour(X, Y, grad.T, zdir='z', offset=gradoffset, colors=gradcolor, \
#                  levels=grcontourlevels, linewidths=gradlinew, extend3d=ext3d)
ax.set_xlabel('W')
ax.set_ylabel('N')
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('$\mathcal{I}(S)$',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
fig.colorbar(surf_r, shrink=cb_shrink, aspect=cb_aspect)

plt.show()

In [40]:
#MEAN VALUES OF INDEXES 1
eps_solmean = np.zeros((Wnsteps,Nnsteps))
ex_solmean = np.zeros((Wnsteps,Nnsteps))

for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    eps_solmean[i,j] = (np.array(Windexes[i,j])).mean()
    ex_solmean[i,j] = (np.array(Pindexes[i,j])).mean()
    
fig = plt.figure(figsize=(10,10))
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

hmin = 1000
hmax = -2000
for coords,d in np.ndenumerate(eps_solmean):
    if math.isnan(d) == False:
        if d < hmin:
            hmin=d
        elif d> hmax:
            hmax=d
hmin *= 1
hmax *= 1

emin = 1000
emax = -2000
for coords,d in np.ndenumerate(ex_solmean):
    if math.isnan(d) == False:
        if d < emin:
            emin=d
        elif d> emax:
            emax=d
emin *= 1
emax *= 1

X, Y = np.meshgrid(Wrange, Nrange)
elev = 15
azimuth = 124
al = 0.7
cb_shrink = 0.5
cb_aspect = 10
label_size = 16
gradoffset = 0
gradlinew = 0.2
ncountourlines = 10
colormap = cm.gnuplot

ax = fig.add_subplot(2, 1, 1, projection='3d')
surf_e = ax.plot_surface(X, Y, ex_solmean.T, rstride=1, cstride=1, alpha=al, cmap=colormap,\
                        vmin=emin,vmax=emax)
ax.set_title('Exact Solution')
ax.set_xlabel('W')
ax.set_ylabel('N')
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('$m(S_{\\text{ex}})$',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
fig.colorbar(surf_e, shrink=cb_shrink, aspect=cb_aspect)


grcontourlevels = list(np.arange(hmin,hmax,(hmax-hmin)/ncountourlines ))

ax = fig.add_subplot(2, 1, 2, projection='3d')
surf_r = ax.plot_surface(X, Y, eps_solmean.T, rstride=1, cstride=1, alpha=al, cmap=colormap,\
                        vmin=hmin,vmax=hmax)
#cset = ax.contour(X, Y, heur_solmean.T, zdir='y', offset=gradoffset, colors=gradcolor, \
#                  levels=grcontourlevels, linewidths=gradlinew, extend3d=ext3d)
ax.set_title('Epsilon Solution')
ax.set_xlabel('W')
ax.set_ylabel('N')
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('$m(S_{\\text{heur}})$',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
fig.colorbar(surf_r, shrink=cb_shrink, aspect=cb_aspect)


plt.show()

In [20]:
#DIFF OF MEAN INDEXES

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

X, Y = np.meshgrid(Wrange, Nrange)
elev = 24
azimuth = -66
al = 1
cb_shrink = 0.5
cb_aspect = 10
label_size = 16
gradoffset = 0
gradlinew = 2
ncountourlines = 10
colormap = cm.gnuplot

fig = plt.figure()
ax = fig.gca(projection='3d')
#diff = ex_solmean.T - heur_solmean.T
diff = eps_solmean.T - ex_solmean.T
dmin = 10
dmax = -20
for coords,d in np.ndenumerate(diff):
    if math.isnan(d) == False:
        if d < dmin:
            dmin=d
        elif d> dmax:
            dmax=d

surf_e = ax.plot_surface(X, Y, diff, rstride=1, cstride=1, alpha=al, cmap=colormap,\
                        vmin=dmin, vmax=dmax)
ax.set_xlabel('W')
ax.set_ylabel('N')
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('$m(S_{\\text{heur}}) - m(S_{\\text{ex}})$',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
fig.colorbar(surf_e, shrink=cb_shrink, aspect=cb_aspect)

plt.show()

NameError: name 'eps_solmean' is not defined

In [53]:
#EPSILON TIMES
matplotlib.style.use('ggplot')

fig = plt.figure(figsize=(10,5))
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

X, Y = np.meshgrid(Wrange, Nrange)
elev = 23
azimuth = -117
al = 0.65
cb_shrink = 0.5
cb_aspect = 10
label_size = 16
#gradoffset = times.min()
linew = 2
#gradcolor = 'k'
ncountourlines = 7
#zmax = 30
ext3d = False
colormap = cm.gnuplot
log = True

def fmttime(x, pos):
    if log == True:
        a = '{:.0f}'.format(np.exp(x) - 1)
    else:
        a = '{:.0f}'.format(x)
    return(a)

if log == True:
    plot_surf = np.log(1+Wtimes)
else:
    plot_surf = Wtimes
    

tmin = Wtimes.min()
tmax = Wtimes.max()
ax = fig.add_subplot(1, 1, 1, projection='3d')
surf_r = ax.plot_surface(X, Y, plot_surf.T, rstride=1, cstride=1, alpha=al, cmap=colormap)
# cset = ax.contour(X, Y, Wtimes.T, zdir='y', offset=0, colors=gradcolor, \
#                   levels=grcontourlevels, linewidths=gradlinew, extend3d=ext3d)
cset = ax.contour(X, Y, plot_surf.T, zdir='y', offset=5, cmap=colormap,linewidths=linew)
cset = ax.contour(X, Y, plot_surf.T, zdir='x', offset=0.005, cmap=colormap,linewidths=linew)
ax.set_xlabel('W')
ax.set_ylabel('N')
#ax.set_zlim(0,0.5)
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('Time (seconds)',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
ax.zaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(fmttime))
fig.colorbar(surf_r, shrink=cb_shrink, aspect=cb_aspect)

plt.show()

In [9]:
#EXACT TIMES
matplotlib.style.use('ggplot')

fig = plt.figure(figsize=(10,5))
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

X, Y = np.meshgrid(Wrange, Nrange)
elev = 35
azimuth = 31
al = 0.65
cb_shrink = 0.5
cb_aspect = 10
label_size = 16
#gradoffset = times.min()
#gradlinew = 3
#gradcolor = 'k'
ncountourlines = 7
#zmax = 30
ext3d = False
colormap = cm.gnuplot

#mingr = grad.min()
#maxgr = grad.max()
#grcontourlevels = list(np.arange(mingr,maxgr,(maxgr-mingr)/ncountourlines ))

tmin = Ptimes.min()
tmax = Ptimes.max()
ax = fig.add_subplot(1, 1, 1, projection='3d')
surf_r = ax.plot_surface(X, Y, Ptimes.T, rstride=1, cstride=1, alpha=al,\
                         vmin=Ptimes.min(),vmax=Ptimes.max(), cmap=colormap)
#cset = ax.contour(X, Y, grad.T, zdir='z', offset=gradoffset, colors=gradcolor, \
#                  levels=grcontourlevels, linewidths=gradlinew, extend3d=ext3d)
ax.set_xlabel('W')
ax.set_ylabel('N')
ax.set_zlim(0,0.2)
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('time',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
fig.colorbar(surf_r, shrink=cb_shrink, aspect=cb_aspect)

plt.show()

In [47]:
#EPSILON NNODES
#matplotlib.style.use('ggplot')

fig = plt.figure(figsize=(10,5))
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

X, Y = np.meshgrid(Wrange, Nrange)
elev = 23
azimuth = -117
al = 0.65
cb_shrink = 0.5
cb_aspect = 10
label_size = 16
#gradoffset = times.min()
linew = 2
#gradcolor = 'k'
ncountourlines = 7
#zmax = 30
ext3d = False
colormap = cm.gnuplot
log = True


def fmt0(x, pos):
    if log == True:
        a = '{:.0e}'.format(np.exp(x) - 1)
    else:
        a = '{:.0e}'.format(x)
    return(a)


if log == True:
    plot_surf = np.log(1+Wnnodes)
else:
    plot_surf = Wnnodes

min_entropy = data.df['plog1onp'].min()
c = np.zeros((Wnsteps,Nnsteps))
for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    N = Nrange[j]
    c[i,j] = 200*(N**2)/(epsilon*min_entropy)

ax = fig.add_subplot(1, 1, 1, projection='3d')
surf_r = ax.plot_surface(X, Y, plot_surf.T, rstride=1, cstride=1, alpha=al, cmap=colormap)
#surf_c = ax.plot_surface(X, Y, c.T, rstride=1, cstride=1, alpha=al/2, color='b')

# cset = ax.contour(X, Y, Wtimes.T, zdir='y', offset=0, colors=gradcolor, \
#                   levels=grcontourlevels, linewidths=gradlinew, extend3d=ext3d)
cset = ax.contour(X, Y, plot_surf.T, zdir='y', offset=5, cmap=colormap,linewidths=linew)
cset = ax.contour(X, Y, plot_surf.T, zdir='x', offset=0.005, cmap=colormap,linewidths=linew)
#cset = ax.contour(X, Y, c.T, zdir='y', offset=4, cmap=colormap,linewidths=linew)
ax.set_xlabel('W')
ax.set_ylabel('N')
#ax.set_zlim(0,0.5)
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('Number of Nodes',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
ax.zaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(fmt0))
cbar = fig.colorbar(surf_r,shrink=cb_shrink, aspect=cb_aspect, \
                    format=matplotlib.ticker.FuncFormatter(fmt1))

plt.show()

In [39]:
W = 0.027500000000000007
N = 5

epsilon = 0.2

data.quantize_entropy(N, epsilon)

prbl_eps = BPV.BPV("decgraphW",data,N,W,time_solver=True,\
                          use_quantized_entropy=True)
prbl_eps.solve()
prbl_eps.pprint_solution()

print(prbl_eps.solution_entropy)
prbl_ex = BPV.BPV("glpk",data,N,W,time_solver=True)
prbl_ex.solve()
prbl_ex.pprint_solution()



Solver =  decgraphW 
Entropy =  0.134172434598 
Cardinality =  5 
Rate =  0.0254217843542
Solution time in seconds =  5.73408278900024
0.134172434598

Solver =  glpk 
Entropy =  0.134172434598 
Cardinality =  5 
Rate =  0.0254217843542
Solution time in seconds =  0.052571870997780934


In [42]:
#Wrange[5], Nrange[0]
#relerr[5,0]
Wentropies[0,5], Pentropies[5,0]

(0.037956838420973367, 0.1341724345982187)